<a href="https://colab.research.google.com/github/RichardHarwood/3D-Cell-Workflow/blob/main/Rootine_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%capture
!pip install wget
!pip install open3d
!pip install vedo
#!pip install stackview ipycanvas==0.11
import os
!git clone https://github.com/wolny/pytorch-3dunet.git
%cd pytorch-3dunet
!python setup.py install

In [4]:
import torch
torch.cuda.is_available()

False

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#need to set appropriate  home folder
import os.path
from os import path
if path.exists('/content/drive/MyDrive/3D_UNET_rootine_eg/') == False:
  os.mkdir('/content/drive/MyDrive/3D_UNET_rootine_eg/')
home_folder = "/content/drive/MyDrive/3D_UNET_rootine_eg/"

In [7]:
#Create Config Folder
if path.exists(home_folder+'config_files') == False:
  os.mkdir(home_folder+'config_files')
config_folder = home_folder+'config_files/'

In [8]:
#Create Checkpoint Folder
if path.exists(home_folder+'checkpoint_dir') == False:
  os.mkdir(home_folder+'checkpoint_dir')
checkpoint_dir = home_folder+'checkpoint_dir/'

In [9]:
#Create Checkpoint Folder
if path.exists(home_folder+'rootine_v2_data_dir/') == False:
  os.mkdir(home_folder+'rootine_v2_data_dir/')
rootine_v2_data_dir = home_folder+'rootine_v2_data_dir/'

In [10]:
#Create Training_data Folder

if path.exists(home_folder+'training_data') == False:
  os.mkdir(home_folder+'training_data')

if path.exists(home_folder+'training_data/train/') == False:
  os.mkdir(home_folder+'training_data/train/')

if path.exists(home_folder+'training_data/qc/') == False:
  os.mkdir(home_folder+'training_data/qc/')

In [11]:
#Create Folder to store images we want to segment
if path.exists(home_folder+'segment_this_folder/') == False:
  os.mkdir(home_folder+'segment_this_folder/')
imgs_to_seg_dir = home_folder+'segment_this_folder/'

Download Data and Mask from Rootine-V2 paper

In [12]:
import wget


if os.path.isfile(home_folder+'rootine_v2_data_dir/rootine_mask.tif'):
  print("already downloaded")
else:
  wget.download("https://zenodo.org/records/13984519/files/"+"rootine_mask.tif"+"?download=1", out=home_folder+'rootine_v2_data_dir/rootine_mask.tif')

if os.path.isfile(home_folder+'rootine_v2_data_dir/rootine_img.tif'):
  print("already downloaded")
else:
  wget.download("https://zenodo.org/records/13984519/files/"+"rootine_raw_scan.tif"+"?download=1", out=home_folder+'rootine_v2_data_dir/rootine_img.tif')

already downloaded
already downloaded


To fit inside Colabs free resources we need to work on a sub sample of the data and create less training data. As such the methods and results are not a complete 1:1 but can be tweaked to match the full dataset (comments will be added on how to run the full data set if Colab resources are purchased or this notebook is being run locally)

Download the orginal 3D_unet model

In [13]:
if os.path.isfile(checkpoint_dir+'3D_Unet_Root_Model.pytorch'):
  print("model already download")
else:
  wget.download("https://zenodo.org/records/13958667/files/"+"3D_Unet_Root_Model.pytorch"+"?download=1", out=checkpoint_dir+'3D_Unet_Root_Model.pytorch')


model already download


In [14]:
#create h5

#Convert to H5
import skimage
#import imagecodecs
import h5py
from scipy.ndimage import zoom
#need to reduce image due to RAM depending on what resources you have
#scale_factor=0.2
img=skimage.io.imread(rootine_v2_data_dir+'rootine_img.tif')
img=img[300:500]
hf = h5py.File(imgs_to_seg_dir+"rootine_200_slices_eg.h5", 'w')
hf.create_dataset('raw', data=img)
hf.close()
print("created h5 file")


created h5 file


In [15]:
model_name = "UNet3D"
in_channels = 1
out_channels= 1
layer_order = "gcr"
f_maps = [32,
          64,
          128,
          256]
num_groups = 8
final_sigmoid = True
is_segmentation = True
batch_size= 2  #important for gpu memory
num_workers= 2
raw_internal_path= "raw"
label_internal_path= "label"
weight_internal_path= None
######################
slice_builder_name= "SliceBuilder"
slice_builder_name_predict = "SliceBuilder"
patch_shape= [40, 170, 170]  #Change depending on GPU memory
stride_shape= [20, 40, 40]   #Change depending on GPU memory
halo_shape= [16, 32, 32]
threshold = 0.01
slack_acceptance = 0.01
name_transformer= "Standardize"
name_transformer_label= "BlobsToMask"
append_label= False
boundary= False
ToTensor_name = "ToTensor"
expand_dims_false= False
expand_dims_true= True
#####################
predictor_name='StandardPredictor'
#################################
file_paths_test= imgs_to_seg_dir
#############################
model_path=checkpoint_dir+'3D_Unet_Root_Model.pytorch'


In [16]:
import yaml
test_config_yaml ={
    "model_path": model_path,
    "model": {
        "name": model_name,
        "in_channels": in_channels,
        "out_channels": out_channels,
        "layer_order": layer_order,
        "f_maps": f_maps,
        "num_groups": num_groups,
        "final_sigmoid": final_sigmoid,
        "is_segmentation": is_segmentation
    },
    "predictor": {
        "name": predictor_name
    },
    "loaders": {
        "batch_size": batch_size,
        "raw_internal_path": raw_internal_path,
        "num_workers": num_workers,
        "test": {
            "file_paths": [
                file_paths_test
            ],
            "slice_builder": {
                "name": slice_builder_name_predict,
                "patch_shape": patch_shape,
                "stride_shape": patch_shape,
                "halo_shape": halo_shape
            },
            "transformer": {
                "raw": [
                    {
                        "name": name_transformer
                    },
                    {
                        "name": ToTensor_name,
                        "expand_dims": expand_dims_true
                    }
                ]
            }
        }
    }
}

with open(config_folder+'predict.yaml', 'w') as yaml_file:
    yaml.dump(test_config_yaml, yaml_file, default_flow_style=False, sort_keys=False)


In [17]:
!predict3dunet --config /content/drive/MyDrive/3D_UNET_rootine_eg/config_files/predict.yaml

2024-11-22 00:18:43,639 [MainThread] WARNING ConfigLoader - CUDA not available, using CPU
2024-11-22 00:18:43,745 [MainThread] INFO UNet3DPredict - Loading model from /content/drive/MyDrive/3D_UNET_rootine_eg/checkpoint_dir/3D_Unet_Root_Model.pytorch...
/usr/local/lib/python3.10/dist-packages/pytorch3dunet-1.8.7-py3.10.egg/pytorch3dunet/unet3d/utils.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializa